In [ ]:
# Import Dependencies
import csv
import pandas as pd
from lxml import html
import requests
import time

# Set Initial Variables
base_url = 'http://www.boxofficemojo.com'

movie_link_list = []
f = open('mojo_links_test.csv')
csv_f = csv.reader(f)
movie_links = list(csv_f)[0]

In [ ]:
# Create variable lists for dictionary
domestic_total_list = []
genre_list = []
budget_list = []
rating_list = []
release_date_list = []
title_list = []
opening_list = []

In [ ]:
# Set iteratetor
i = 0

# Loop through csv movie links
for movie in movie_links:

    # Error Handling     
    try:
        page = requests.get(base_url+movie)
        tree = html.fromstring(page.text)
    
        movie_clean = movie.replace("/movies/?page=main&id=","")
        movie_title = movie_clean.replace(".htm","")
    
        # Domestic Total Gross
        domestice_total = tree.xpath('//*[@id="body"]//font[starts-with(normalize-space(.),"Domestic Total Gross:")]/b/text()')
        # Genre
        genre = tree.xpath('//*[@id="body"]//td[starts-with(normalize-space(.),"Genre:")]/b/text()')
        # Production Budget
        budget = tree.xpath('//*[@id="body"]//td[starts-with(normalize-space(.),"Production Budget:")]/b/text()')
        # MPAA Rating
        rating = tree.xpath('//*[@id="body"]//td[starts-with(normalize-space(.),"MPAA Rating:")]/b/text()')
        # Release Date
        release_date = tree.xpath('//*[@id="body"]//td[starts-with(normalize-space(.),"Release Date:")]/b/nobr/a/text()')

        # Work around &nbsp; issue
        tree = html.fromstring(page.text.replace("&nbsp;", ""))
        # Opening Weekend
        opening_weekend = tree.xpath('//*[@class="mp_box_content"]//a[starts-with(normalize-space(.),"OpeningWeekend:")]/../following-sibling::*/text()')
        # Title
        title = tree.xpath('/html/head/title/text()')
        
        # convert values from list to string
        domestic_total_list.append(''.join(domestice_total))
        genre_list.append(''.join(genre))
        budget_list.append(''.join(budget))
        rating_list.append(''.join(rating))
        release_date_list.append(''.join(release_date))
        title_list.append(''.join(title))
        opening_list.append(''.join(opening_weekend))
        print(i)
        i += 1
        
    except requests.exceptions.ConnectionError:
       print("Connection refused")

In [ ]:
movie_data = {
    "domestic_total":domestic_total_list,
    "genre":genre_list,
    "budget":budget_list,
    "rating":rating_list,
    "release_date":release_date_list,
    "title":title_list,
    "opening_total":opening_list
}

df_movie_data = pd.DataFrame(movie_data)
df_movie_data

In [ ]:
df_movie_data.to_csv('mojo_data.csv',index=False)